Artilces:<br/>
[Perceptrons](https://direct.mit.edu/books/edited-volume/5431/chapter-abstract/3958520/1969-Marvin-Minsky-and-Seymour-Papert-Perceptrons?redirectedFrom=PDF)<br/>
[The Organization of Behavior](https://pubmed.ncbi.nlm.nih.gov/10643472/)<br/>
[Learning Internal Representations by Error Propagation](https://www.semanticscholar.org/paper/Learning-internal-representations-by-error-Rumelhart-Hinton/319f22bd5abfd67ac15988aa5c7f705f018c3ccd)<br/>
[A logical calculus of the ideas immanent in nervous activity](https://link.springer.com/article/10.1007/BF02478259)<br/>
[The perceptron: A probabilistic model for information storage and organization in the brain](https://www.semanticscholar.org/paper/The-perceptron%3A-a-probabilistic-model-for-storage-Rosenblatt/5d11aad09f65431b5d3cb1d85328743c9e53ba96)<br/>


